## Citi Bike Tripdata CSV File Cleanup
This notebook is used to convert a tripdata CSV file from either before or after Feb/2021 (there was a format change that month) <br>
into a modified version of the newer format that is easier for Tableau to handle.<br><br>
This notebook also performs several cleaning operations:
* Renaming column headers to match new format
* Making sure that all stations have the same names, latitudes, and longitudes as their matching station in the 'station_information.json'.
* Changing the usertype attributes to the newer member_casual attributes.
* Adding calculated fields for trip minutes and trip miles.
* Dropping fields that are no longer tracked.
* Making sure the fields are the correct data type.
* Making sure the lats and lngs have the correct number of decimal places.
* Making sure that stations don't have multiple IDs refering to them.
* This notebook also adds new entries into the Station Information file as they are discovered.

### Step 1:
#### IMPORTANT -  Before starting, you must provide the names of the following files into the cell below:
* The CSV file that contains the Citi Bike Trip Data
* the JSON file that contains the bike stations' information

In [231]:
# Name of Citi Bike Trip Data CSV file:
csvTripDataFile = 'JC-202112-citibike-tripdata.csv'

# Name of Citi Bike Station Information JSON file:
jsonStationsFile = 'station_information.json'

# Name of updated Citi Bike Station Information CSV file (it will be created if it doesn't already exist):
csvStationsFile = 'station_information.csv'

### Step 2:

In [232]:
# Import external dependencies:
import pandas as pd
import numpy as np
import json

# Read in Citi Bike Station Information CSV file (or JSON file if it doesn't exist):
successful = False
try:
    DF1 = pd.read_csv(csvStationsFile, low_memory=False)
    successful = True
except:
    try:
        with open(jsonStationsFile) as f:
            jsonData = json.load(f)
            DF1 = pd.DataFrame.from_dict(jsonData['data']['stations'])
            successful = True
            #print(json.dumps(jsonData['data']['stations'], indent=4, sort_keys=True))
            DF1.drop(['electric_bike_surcharge_waiver', 'station_id', 'eightd_has_key_dispenser'], axis=1, inplace=True)
            DF1.drop(['eightd_station_services', 'external_id', 'rental_uris'], axis=1, inplace=True)
    except:
        print("An error occured.\n" + 
              "Please make sure that the files indicated in Step 1 above exist in this directory and are spelled correctly.")

if successful:
    
    # Rearrange and rename columns:
    DF1.rename(columns = {'short_name': 'new_id', 'legacy_id': 'old_id', 'lat': 'latitude', 'lon': 'longitude'}, inplace=True)
    DF1 = DF1[['new_id', 'old_id', 'name', 'region_id','latitude', 'longitude', 'station_type','capacity', 'has_kiosk', 'rental_methods']]

    # Correct data types to enable merging:
    DF1 = DF1.astype({"new_id": 'str', "old_id": 'str'})      
    
    # Round the number of decimal places to 6 for latitude and longitude:
    DF1['latitude'].astype(float).round(decimals=6)
    DF1['longitude'].astype(float).round(decimals=6)
    
    # Display:
    DF1.info()
    print()
    
DF1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1660 entries, 0 to 1659
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   new_id          1660 non-null   object 
 1   old_id          1660 non-null   object 
 2   name            1660 non-null   object 
 3   region_id       1660 non-null   int64  
 4   latitude        1660 non-null   float64
 5   longitude       1660 non-null   float64
 6   station_type    1660 non-null   object 
 7   capacity        1660 non-null   int64  
 8   has_kiosk       1660 non-null   bool   
 9   rental_methods  1660 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 118.5+ KB



,new_id,old_id,name,region_id,latitude,longitude,station_type,capacity,has_kiosk,rental_methods
0,6926.01,72,W 52 St & 11 Ave,71,40.767272,-73.993929,classic,55,True,"['KEY', 'CREDITCARD']"
1,5430.08,79,Franklin St & W Broadway,71,40.719116,-74.006667,classic,33,True,"['KEY', 'CREDITCARD']"
2,5167.06,82,St James Pl & Pearl St,71,40.711174,-74.000165,classic,27,True,"['KEY', 'CREDITCARD']"
3,4354.07,83,Atlantic Ave & Fort Greene Pl,71,40.683826,-73.976323,classic,62,True,"['KEY', 'CREDITCARD']"
4,6148.02,116,W 17 St & 8 Ave,71,40.741776,-74.001497,classic,50,True,"['KEY', 'CREDITCARD']"


### Step 3:

In [233]:
# Read in Citi Bike Trip Data CSV file and convert to dataframe:
DF2 = pd.read_csv(csvTripDataFile, low_memory=False)

# Display column names/types:
DF2.info()
print()

# Determine if the CSV file has the old or new format and display user message:
if 'tripduration' in DF2.columns:
    old_format = True
    print(f"This file has {len(DF2):,} rows in the older format.  Please run Step 4a below.")
else:
    old_format = False
    print(f"This file has {len(DF2):,} rows in the newer format.  Please skip to Step 5a below.")

print()
DF2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48021 entries, 0 to 48020
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ride_id             48021 non-null  object 
 1   rideable_type       48021 non-null  object 
 2   started_at          48021 non-null  object 
 3   ended_at            48021 non-null  object 
 4   start_station_name  48021 non-null  object 
 5   start_station_id    48021 non-null  object 
 6   end_station_name    47662 non-null  object 
 7   end_station_id      47662 non-null  object 
 8   start_lat           48021 non-null  float64
 9   start_lng           48021 non-null  float64
 10  end_lat             47875 non-null  float64
 11  end_lng             47875 non-null  float64
 12  member_casual       48021 non-null  object 
dtypes: float64(4), object(9)
memory usage: 4.8+ MB

This file has 48,021 rows in the newer format.  Please skip to Step 5a below.



,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,DE8FDF918B766B34,classic_bike,2021-12-22 18:44:12,2021-12-22 18:49:56,Marin Light Rail,JC013,Warren St,JC006,40.714584,-74.042817,40.721124,-74.038051,member
1,DFFA588D5D188F27,classic_bike,2021-12-11 12:32:58,2021-12-11 12:57:13,Warren St,JC006,Warren St,JC006,40.721124,-74.038051,40.721124,-74.038051,member
2,3A2A9CA5E1CC449A,classic_bike,2021-12-18 11:08:40,2021-12-18 11:21:10,Warren St,JC006,Warren St,JC006,40.721124,-74.038051,40.721124,-74.038051,member
3,72216E8C562A4E02,classic_bike,2021-12-11 17:28:08,2021-12-11 17:42:16,5 Corners Library,JC018,Warren St,JC006,40.734961,-74.059503,40.721124,-74.038051,member
4,A7B30B3C966FD8A1,classic_bike,2021-12-11 10:39:41,2021-12-11 10:54:56,City Hall - Washington St & 1 St,HB105,Warren St,JC006,40.737360,-74.030970,40.721124,-74.038051,casual


### Step 4a:  (Run this if the CSV file has the older format.)

In [234]:
# If the CSV file has the newer format, alert user and exit, otherwise run this cell:
if not old_format:
    print("This file has the newer format.  Please run Step 5a below.")
else:
    
    # Set some counting variables:
    append_count = 0
    correction_count = 0
    
    # Correct data types to enable accurate searching:
    DF2 = DF2.astype({"start station id": 'str', "end station id": 'str'}) 
    
    # Drop rows with duplicated start station ids:
    DFtemp = DF2.drop_duplicates(subset=['start station id']).dropna(subset=['start station id'])
    
    # Search through all rows for start station ids that do not exist in the station information json:
    for index, row in DFtemp.iterrows():
        if str(row[3]) not in DF1['old_id'].values:
            
            # If the start station name does not exist in the station information json, append this station to it:
            if str(row[4]) not in DF1['name'].values:
                
                # But only append if the name, lat, and lng are not missing:
                if not pd.isnull([row[4], row[5], row[6]]).any():
                    new_row = {
                        'new_id': str(row[3]),
                        'old_id': str(row[3]),
                        'name': str(row[4]),
                        'region_id': 71,
                        'latitude': round(float(row[5]), 6),
                        'longitude': round(float(row[6]), 6),
                        'station_type': 'classic',
                        'capacity': 0,
                        'has_kiosk': True,
                        'rental_methods': '[KEY, CREDITCARD]'
                    }
                    DF1 = DF1.append(new_row, ignore_index=True)
                    append_count += 1
            
            # Otherwise, change the start station id to match the entry in the station information json with the same name:
            else:
                DF2.loc[DF2['start station name']==str(row[4]), ['start station id']] = DF1.loc[DF1['name']==str(row[4]), ['old_id']]
                correction_count += 1
    
    # Drop rows with duplicated end station ids:
    DFtemp = DF2.drop_duplicates(subset=['end station id']).dropna(subset=['end station id'])
    
    # Search through all rows for end station ids that do not exist in the station information json:
    for index, row in DFtemp.iterrows():
        if str(row[7]) not in DF1['old_id'].values:
            
            # If the end station name does not exist in the station information json, append this station to it:
            if str(row[8]) not in DF1['name'].values:
                
                # But only append if the name, lat, and lng are not missing:
                if not pd.isnull([row[8], row[9], row[10]]).any():
                    new_row = {
                        'new_id': str(row[7]),
                        'old_id': str(row[7]),
                        'name': str(row[8]),
                        'region_id': 71,
                        'latitude': round(float(row[9]), 6),
                        'longitude': round(float(row[10]), 6),
                        'station_type': 'classic',
                        'capacity': 0,
                        'has_kiosk': True,
                        'rental_methods': '[KEY, CREDITCARD]'
                    }
                    DF1 = DF1.append(new_row, ignore_index=True)
                    append_count += 1
            
            # Otherwise, change the end station id to match the entry in the station information json with the same name:
            else:
                DF2.loc[DF2['end station name']==str(row[8]), ['end station id']] = DF1.loc[DF1['name']==str(row[8]), ['old_id']]
                correction_count += 1
    
    # Display:
    print(f"{append_count} entries added to {csvStationsFile}.")
    print(f"{correction_count} station IDs corrected in the {csvTripDataFile.split('-')[0]} data set.")
    print("Please run Step 4b below.")


This file has the newer format.  Please run Step 5a below.


### Step 4b:  (Run this if the CSV file has the older format.)

In [235]:
# If the CSV file has the newer format, alert user and exit, otherwise run this cell:
if not old_format:
    print("This file has the newer format.  Please run Step 5a below.")
else:

    # If any rows are missing ending station info, assume the bikes were returned to the starting station:
    DF2['end station id'].fillna(DF2['start station id'], inplace=True)
    DF2['end station name'].fillna(DF2['start station name'], inplace=True)
    DF2['end station latitude'].fillna(DF2['start station latitude'], inplace=True)
    DF2['end station longitude'].fillna(DF2['start station longitude'], inplace=True)
    
    # Correct data types to enable merging:
    DF2 = DF2.astype({"start station id": 'str', "end station id": 'str'})    

    # Replace all starting stations' latitudes, longitudes, and names with the official lats, lons, and names from JSON dataframe:
    DF3 = pd.merge(DF2, DF1, left_on='start station id', right_on='old_id', how='left')
    DF3['start_lat'] = DF3['latitude']
    DF3['start_lng'] = DF3['longitude']
    DF3['start_station_name'] = DF3['name']
    DF3['start_station_id'] = DF3['new_id']
    DF3.drop(['new_id', 'old_id', 'name', 'region_id', 'latitude', 'longitude'], axis=1, inplace=True) 
    DF3.drop(['station_type', 'capacity', 'has_kiosk', 'rental_methods'], axis=1, inplace=True)

    # Replace all ending stations' latitudes, longitudes, and names with the official lats, lons, and names from JSON dataframe:
    DF3 = pd.merge(DF3, DF1, left_on='end station id', right_on='old_id', how='left')
    DF3['end_lat'] = DF3['latitude']
    DF3['end_lng'] = DF3['longitude']
    DF3['end_station_name'] = DF3['name']
    DF3['end_station_id'] = DF3['new_id']
    DF3.drop(['new_id', 'old_id', 'name', 'region_id', 'latitude', 'longitude'], axis=1, inplace=True) 
    DF3.drop(['station_type', 'capacity', 'has_kiosk', 'rental_methods'], axis=1, inplace=True)
    
    # Fill NaN values in new columns with old values from the original dataframe:
    DF3['start_lat'] = DF3['start_lat'].fillna(DF3['start station latitude'])
    DF3['start_lng'] = DF3['start_lng'].fillna(DF3['start station longitude'])
    DF3['start_station_name'] = DF3['start_station_name'].fillna(DF3['start station name'])
    DF3['end_lat'] = DF3['end_lat'].fillna(DF3['end station latitude'])
    DF3['end_lng'] = DF3['end_lng'].fillna(DF3['end station longitude'])
    DF3['end_station_name'] = DF3['end_station_name'].fillna(DF3['end station name'])
    
    # Drop any remaining rows containing NaN values (if any):
    DF3.dropna()

    # Drop unwanted columns:
    DF3.drop(['tripduration', 'bikeid', 'birth year', 'gender'], axis=1, inplace=True)
    DF3.drop(['start station name', 'start station id', 'start station latitude', 'start station longitude'], axis=1, inplace=True)
    DF3.drop(['end station name', 'end station id', 'end station latitude', 'end station longitude'], axis=1, inplace=True)

    # Rename columns:
    DF3.rename(columns = {'starttime':'started_at', 'stoptime':'ended_at'}, inplace=True)
    DF3.rename(columns = {'usertype':'member_casual'}, inplace=True)

    # Replace the older attributes in member_casual field with newer attributes:
    DF3.loc[DF3['member_casual'] == 'Subscriber', 'member_casual'] = 'member'
    DF3.loc[DF3['member_casual'] == 'Customer', 'member_casual'] = 'casual'

    # Create Trip ID column:
    DF3['trip_id'] = csvTripDataFile.split("-")[0] + DF3.index.map(str)
    
    # Round the number of decimal places to 6 for lats and lngs:
    DF3['start_lat'].astype(float).round(decimals=6)
    DF3['start_lng'].astype(float).round(decimals=6)
    DF3['end_lat'].astype(float).round(decimals=6)
    DF3['end_lng'].astype(float).round(decimals=6)

    # Display
    print("Dataframe successfully cleaned.")
    print(f"{len(DF3):,} rows processed.")


This file has the newer format.  Please run Step 5a below.


### Step 5a: (Run this if the CSV file has the newer format.)

In [236]:
# If the CSV file has the newer format, alert user and exit, otherwise run this cell:
if old_format:
    print("This file has the older format.  Please run Step 4a above.")
else:
    
    # Set some counting variables:
    append_count = 0
    correction_count = 0
    
    # Correct data types to enable accurate searching:
    DF2 = DF2.astype({"start_station_id": 'str', "end_station_id": 'str'}) 
    
    # Drop rows with duplicated start station ids:
    DFtemp = DF2.drop_duplicates(subset=['start_station_id']).dropna(subset=['start_station_id'])
    
    # Search through all rows for start station ids that do not exist in the station information json:
    for index, row in DFtemp.iterrows():
        if str(row[5]) not in DF1['new_id'].values:
            
            # If the start station name does not exist in the station information json, append this station to it:
            if str(row[4]) not in DF1['name'].values:

                # But only append if the name, lat, and lng are not missing:
                if not pd.isnull([row[4], row[8], row[9]]).any():                
                    new_row = {
                        'new_id': str(row[5]),
                        'old_id': str(row[5]),
                        'name': str(row[4]),
                        'region_id': 71,
                        'latitude': round(float(row[8]), 6),
                        'longitude': round(float(row[9]), 6),
                        'station_type': 'docked_bike',
                        'capacity': 0,
                        'has_kiosk': True,
                        'rental_methods': '[KEY, CREDITCARD]'
                    }
                    DF1 = DF1.append(new_row, ignore_index=True)
                    append_count += 1
            
            # Otherwise, change the start station id to match the entry in the station information json with the same name:
            else:
                DF2.loc[DF2['start_station_name']==str(row[4]), ['start_station_id']] = DF1.loc[DF1['name']==str(row[4]), ['new_id']]
                correction_count += 1
    
    # Drop rows with duplicated end station ids:
    DFtemp = DF2.drop_duplicates(subset=['end_station_id']).dropna(subset=['end_station_id'])
    
    # Search through all rows for end station ids that do not exist in the station information json:
    for index, row in DFtemp.iterrows():
        if str(row[7]) not in DF1['new_id'].values:
            
            # If the end station name does not exist in the station information json, append this station to it:
            if str(row[6]) not in DF1['name'].values:
                
                # But only append if the name, lat, and lng are not missing:
                if not pd.isnull([row[6], row[10], row[11]]).any():                
                    new_row = {
                        'new_id': str(row[7]),
                        'old_id': str(row[7]),
                        'name': str(row[6]),
                        'region_id': 71,
                        'latitude': round(float(row[10]), 6),
                        'longitude': round(float(row[11]), 6),
                        'station_type': 'docked_bike',
                        'capacity': 0,
                        'has_kiosk': True,
                        'rental_methods': '[KEY, CREDITCARD]'
                    }
                    DF1 = DF1.append(new_row, ignore_index=True)
                    append_count += 1
            
            # Otherwise, change the end station id to match the entry in the station information json with the same name:
            else:
                DF2.loc[DF2['end_station_name']==str(row[6]), ['end_station_id']] = DF1.loc[DF1['name']==str(row[6]), ['new_id']]
                correction_count += 1
    
    # Display:
    print(f"{append_count} entries added to {csvStationsFile}.")
    print(f"{correction_count} station IDs corrected in the {csvTripDataFile.split('-')[0]} data set.")
    print("Please run Step 5b below.")


0 entries added to station_information.csv.
0 station IDs corrected in the JC data set.
Please run Step 5b below.


### Step 5b: (Run this if the CSV file has the newer format.)

In [237]:
# If the CSV file has the older format, alert user and exit, otherwise run this cell:
if old_format:
    print("This file has the older format.  Please run Step 4a above.")
else:

    # If any rows are missing ending station info, assume the bikes were returned to the starting station:
    DF2['end_station_id'].fillna(DF2['start_station_id'], inplace=True)
    DF2['end_station_name'].fillna(DF2['start_station_name'], inplace=True)
    DF2['end_lat'].fillna(DF2['start_lat'], inplace=True)
    DF2['end_lng'].fillna(DF2['start_lng'], inplace=True)

    # Correct data types to enable merging:
    DF2 = DF2.astype({"start_station_id": 'str', "end_station_id": 'str'})
    
    # Rename columns:
    DF2.rename(columns = {'start_station_name':'start station name', 'end_station_name':'end station name'}, inplace=True)
    DF2.rename(columns = {'start_station_id':'start station id', 'end_station_id':'end station id'}, inplace=True)
    DF2.rename(columns = {'start_lat':'start station latitude', 'start_lng':'start station longitude'}, inplace=True)    
    DF2.rename(columns = {'end_lat':'end station latitude', 'end_lng':'end station longitude'}, inplace=True)
    DF2.rename(columns = {'ride_id': 'trip_id'}, inplace=True)
    
    # Replace all starting stations' latitudes, longitudes, and names with the official lats, lons, and names from JSON dataframe:
    DF3 = pd.merge(DF2, DF1, left_on='start station id', right_on='new_id', how='left')
    DF3['start_lat'] = DF3['latitude']
    DF3['start_lng'] = DF3['longitude']
    DF3['start_station_name'] = DF3['name']
    DF3['start_station_id'] = DF3['new_id']
    DF3.drop(['new_id', 'old_id', 'name', 'region_id', 'latitude', 'longitude'], axis=1, inplace=True) 
    DF3.drop(['station_type', 'capacity', 'has_kiosk', 'rental_methods'], axis=1, inplace=True)

    # Replace all ending stations' latitudes, longitudes, and names with the official lats, lons, and names from JSON dataframe:
    DF3 = pd.merge(DF3, DF1, left_on='end station id', right_on='new_id', how='left')
    DF3['end_lat'] = DF3['latitude']
    DF3['end_lng'] = DF3['longitude']
    DF3['end_station_name'] = DF3['name']
    DF3['end_station_id'] = DF3['new_id']
    DF3.drop(['new_id', 'old_id', 'name', 'region_id', 'latitude', 'longitude'], axis=1, inplace=True) 
    DF3.drop(['station_type', 'capacity', 'has_kiosk', 'rental_methods'], axis=1, inplace=True)
    
    # Fill NaN values in new columns with old values from the original dataframe:
    DF3['start_lat'] = DF3['start_lat'].fillna(DF3['start station latitude'])
    DF3['start_lng'] = DF3['start_lng'].fillna(DF3['start station longitude'])
    DF3['start_station_name'] = DF3['start_station_name'].fillna(DF3['start station name'])
    DF3['start_station_id'] = DF3['start_station_id'].fillna(DF3['start station id'])
    DF3['end_lat'] = DF3['end_lat'].fillna(DF3['end station latitude'])
    DF3['end_lng'] = DF3['end_lng'].fillna(DF3['end station longitude'])
    DF3['end_station_name'] = DF3['end_station_name'].fillna(DF3['end station name'])
    DF3['end_station_id'] = DF3['end_station_id'].fillna(DF3['end station id'])
    
    # Drop any remaining rows containing NaN values (if any):
    DF3.dropna()
    
    # Drop unwanted columns:
    DF3.drop(['rideable_type'], axis=1, inplace=True)
    DF3.drop(['start station name', 'start station latitude', 'start station longitude'], axis=1, inplace=True)
    DF3.drop(['end station name', 'end station latitude', 'end station longitude'], axis=1, inplace=True)
 
    # Round the number of decimal places to 6 for lats and lngs:
    DF3['start_lat'].astype(float).round(decimals=6)
    DF3['start_lng'].astype(float).round(decimals=6)
    DF3['end_lat'].astype(float).round(decimals=6)
    DF3['end_lng'].astype(float).round(decimals=6)

    # Display
    print("Dataframe successfully cleaned.")
    print(f"{len(DF3):,} rows processed.")


Dataframe successfully cleaned.
48,021 rows processed.


### Step 6:

In [238]:
# Add trip duration column:
DF3['trip_minutes'] = (pd.to_datetime(DF3['ended_at']) - pd.to_datetime(DF3['started_at'])).astype('timedelta64[m]').astype(int)

# Add trip distance column:
latdeg2miles = 69   # degrees lat to miles conversion factor for NYC
lngdeg2miles = 53   # degrees lon to miles conversion factor for NYC
DF3['trip_miles'] = np.sqrt(((DF3['end_lng'] - DF3['start_lng']) * lngdeg2miles)**2 + ((DF3['end_lat'] -DF3['start_lat']) * latdeg2miles)**2).round(decimals=2)

# Rearrange the columns:
DF3 = DF3[['trip_id', 'member_casual', 'started_at', 'start_station_id', 'start_station_name', 'start_lat', 'start_lng',
           'ended_at', 'end_station_id', 'end_station_name', 'end_lat', 'end_lng', 'trip_minutes', 'trip_miles']]

# Display
DF3.info()
print()
print(f"Final Cleaned Citi Bike Trip Data Table ({len(DF3):,} Rows):")
DF3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48021 entries, 0 to 48020
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   trip_id             48021 non-null  object 
 1   member_casual       48021 non-null  object 
 2   started_at          48021 non-null  object 
 3   start_station_id    48021 non-null  object 
 4   start_station_name  48021 non-null  object 
 5   start_lat           48021 non-null  float64
 6   start_lng           48021 non-null  float64
 7   ended_at            48021 non-null  object 
 8   end_station_id      48021 non-null  object 
 9   end_station_name    48021 non-null  object 
 10  end_lat             48021 non-null  float64
 11  end_lng             48021 non-null  float64
 12  trip_minutes        48021 non-null  int32  
 13  trip_miles          48021 non-null  float64
dtypes: float64(5), int32(1), object(8)
memory usage: 5.3+ MB

Final Cleaned Citi Bike Trip Data Table (48,

,trip_id,member_casual,started_at,start_station_id,start_station_name,start_lat,start_lng,ended_at,end_station_id,end_station_name,end_lat,end_lng,trip_minutes,trip_miles
0,DE8FDF918B766B34,member,2021-12-22 18:44:12,JC013,Marin Light Rail,40.714584,-74.042817,2021-12-22 18:49:56,JC006,Warren St,40.721124,-74.038051,5,0.52
1,DFFA588D5D188F27,member,2021-12-11 12:32:58,JC006,Warren St,40.721124,-74.038051,2021-12-11 12:57:13,JC006,Warren St,40.721124,-74.038051,24,0.00
2,3A2A9CA5E1CC449A,member,2021-12-18 11:08:40,JC006,Warren St,40.721124,-74.038051,2021-12-18 11:21:10,JC006,Warren St,40.721124,-74.038051,12,0.00
3,72216E8C562A4E02,member,2021-12-11 17:28:08,JC018,5 Corners Library,40.734961,-74.059503,2021-12-11 17:42:16,JC006,Warren St,40.721124,-74.038051,14,1.48
4,A7B30B3C966FD8A1,casual,2021-12-11 10:39:41,HB105,City Hall - Washington St & 1 St,40.737360,-74.030970,2021-12-11 10:54:56,JC006,Warren St,40.721124,-74.038051,15,1.18


### Step 7:

In [239]:
# Save the cleaned and modified Citi Bike Trip Data as a CSV file:
outputFile = csvTripDataFile.split(".")[0] + "-cleaned.csv"
DF3.to_csv(outputFile, index=False)

# Save the cleaned and modified Citi Bike Station Information Data as a CSV file:
DF1.to_csv(csvStationsFile, index=False)

# Display user message:
print(f"{outputFile} successfully created.")
print(f"{csvStationsFile} successfully created/updated.")

JC-202112-citibike-tripdata-cleaned.csv successfully created.
station_information.csv successfully created/updated.


In [240]:
duplicates = DF1[DF1.duplicated(['old_id', 'new_id', 'name'])]
duplicates

,new_id,old_id,name,region_id,latitude,longitude,station_type,capacity,has_kiosk,rental_methods
